In [1]:
import pandas as pd
import graphviz
from sklearn import tree
from sklearn.model_selection import train_test_split
import numpy as np


DATA_PATH = "附件一：325个样本数据.xlsx"

df = pd.read_excel(DATA_PATH)

#将数据划分为标签和特征
# = df.drop(['辛烷值RON'],axis = 1)
X = df.iloc[:,2:]
y = df['辛烷值RON']

# 分训练集和测试集
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.3)


from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(
    n_estimators=100,
    random_state=1,
    n_jobs=-1)
forest.fit(Xtrain, Ytrain)

Ypredict = forest.predict(Xtest)




In [42]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import graphviz
from sklearn import tree
from sklearn.model_selection import train_test_split
import numpy as np
 
 
class PSO:
    def __init__(self, parameters):
        """
        particle swarm optimization
        parameter: a list type, like [NGEN, pop_size, var_num_min, var_num_max]
        """
        # 初始化
        self.NGEN = parameters[0]    # 迭代的代数
        self.pop_size = parameters[1]    # 种群大小
        self.var_num = len(parameters[2])    # 变量个数
        self.bound = []                 # 变量的约束范围
        self.bound.append(parameters[2])
        self.bound.append(parameters[3])
 
        self.pop_x = np.zeros((self.pop_size, self.var_num))    # 所有粒子的位置
        self.pop_v = np.zeros((self.pop_size, self.var_num))    # 所有粒子的速度
        self.p_best = np.zeros((self.pop_size, self.var_num))   # 每个粒子最优的位置
        self.g_best = np.zeros((1, self.var_num))   # 全局最优的位置
 
        # 初始化第0代初始全局最优解
        temp = 0.3
        for i in range(self.pop_size):
            for j in range(self.var_num):
                
                #self.pop_x[i][j] = df.iloc[0,:].values[j + 6]
                self.pop_x[i][j] = random.uniform(self.bound[0][j], self.bound[1][j])
                self.pop_v[i][j] = random.uniform(-parameters[5][j], parameters[5][j])
            
            self.p_best[i] = self.pop_x[i]      # 储存最优的个体
            fit = self.fitness(np.append(parameters[4],self.p_best[i]).reshape(1, -1))
            
            if fit < temp and fit> 0:
                self.g_best = self.p_best[i]
                temp = fit
                
    def fitness(self, ind_var):
        """
        个体适应值计算
        """
        X = df.iloc[:,2:29].values
        y = df['辛烷值RON'].values

        # 分训练集和测试集
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.3)

        from sklearn.ensemble import RandomForestRegressor
        forest = RandomForestRegressor(
                        n_estimators=100,
                        random_state=1,
                        n_jobs=-1)
        forest.fit(Xtrain, Ytrain)

        y_predict = forest.predict(ind_var)
        
        y = (df.iloc[0,0] - y_predict)/df.iloc[0,0]
        
        return y
 
    def update_operator(self, pop_size):
        """
        更新算子：更新下一时刻的位置和速度
        """
        c1 = 2     # 学习因子，一般为2
        c2 = 2
        w = 1.5    # 自身权重因子
        for i in range(pop_size):
            
            # 更新速度
            self.pop_v[i] = w * self.pop_v[i] + c1 * random.uniform(0, 1) * (
                        self.p_best[i] - self.pop_x[i]) + c2 * random.uniform(0, 1) * (self.g_best - self.pop_x[i])
            # 更新位置
            self.pop_x[i] = self.pop_x[i] + self.pop_v[i]
            # 越界保护
            for j in range(self.var_num):
                if self.pop_x[i][j] < self.bound[0][j]:
                    self.pop_x[i][j] = self.bound[0][j]
                if self.pop_x[i][j] > self.bound[1][j]:
                    self.pop_x[i][j] = self.bound[1][j]
            # 更新p_best和g_best
            if self.fitness(np.append(parameters[4],self.pop_x[i]).reshape(1, -1)) > self.fitness(np.append(parameters[4],self.p_best[i]).reshape(1, -1)):
                self.p_best[i] = self.pop_x[i]
            if self.fitness(np.append(parameters[4],self.pop_x[i]).reshape(1, -1)) > self.fitness(np.append(parameters[4],self.g_best).reshape(1, -1)):
                self.g_best = self.pop_x[i]
 
    def main(self):
        popobj = []
        self.ng_best = np.zeros((1, self.var_num))[0]
        for gen in range(self.NGEN):
            self.update_operator(self.pop_size)
            popobj.append(self.fitness(np.append(parameters[4],self.g_best).reshape(1, -1)))
            print('############ Generation {} ############'.format(str(gen + 1)))
            if self.fitness(np.append(parameters[4],self.g_best).reshape(1, -1)) > self.fitness(np.append(parameters[4],self.ng_best).reshape(1, -1)):
                self.ng_best = self.g_best.copy()
            print('最好的位置：{}'.format(np.append(parameters[4],self.ng_best)))
            print('最小的函数值：{}'.format(self.fitness(np.append(parameters[4],self.ng_best).reshape(1, -1))))
        print("---- End of (successful) Searching ----")
 
        plt.figure()
        plt.title("Figure1")
        plt.xlabel("iterators", size=14)
        plt.ylabel("fitness", size=14)
        t = [t for t in range(self.NGEN)]
        plt.plot(t, popobj, color='b', linewidth=2)
        plt.show()
 
 
if __name__ == '__main__':
    
    DATA_PATH = "附件一：325个样本数据.xlsx"
    df = pd.read_excel(DATA_PATH)

    DATA_PATH = "附件四：354个操作变量信息.xlsx"
    df1 = pd.read_excel(DATA_PATH)
    
    NGEN = 60
    popsize = 20
    low = df1.iloc[0:23,1].values  # 解空间范围 
    up = df1.iloc[0:23,2].values
    r = df1.iloc[0:23,3].values
    frist = df.iloc[0,2:6].values
    parameters = [NGEN, popsize, low, up, frist,r,df]
    pso = PSO(parameters)
    pso.main()

KeyboardInterrupt: 